# Name: Soh Jian Min
# Admin Number: P2238856
# Class: EL/EP0302/FT/01
# Title: Market Trends and Insights for Housing Agents in Singapore: Analyzing Popular Flat Types, Sales, and Economic Influences
---
## Story Line
As a housing agent in Singapore, understanding market trends and identifying the most popular types of flats and sales is crucial for advising clients and making informed decisions.

## Questions
1. What types of sales are most popular in the private residential property market?
2. Which flat types are most popular in the HDB resale market?
3. Does GDP affect the price index and number of transactions/applications?
4. What types of flats should housing agents focus on to maximize their sales?
5. How does the distribution of the resale price index compare to the private residential property price index over time?

## Data Source
1. https://beta.data.gov.sg/datasets/d_7c69c943d5f0d89d6a9a773d2b51f337/view (Private Residential Property Transactions in the Whole of Singapore, Quarterly)
2. https://beta.data.gov.sg/datasets/d_02aa4bb51bc674f3a2d0b9bb6911d934/view (Number of Resale Applications Registered by Flat Type)
3. https://beta.data.gov.sg/datasets/d_14f63e595975691e7c24a27ae4c07c79/view (Housing And Development Board Resale Price Index (1Q2009 = 100), Quarterly)
4. https://beta.data.gov.sg/datasets/d_97f8a2e995022d311c6c68cfda6d034c/view (Private Residential Property Price Index (Base Quarter 2009-Q1 = 100), Quarterly)
5. https://beta.data.gov.sg/datasets/d_dce7d88c668273bb8c1291027e63325a/view (Per Capita GNI And Per Capita GDP At Current Prices, Annual)

## Load data from CSV files

In [104]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load datasets
private_sales = pd.read_csv('PrivateResidentialPropertyTransactionsintheWholeofSingaporeQuarterly.csv')
hdb_resale = pd.read_csv('NumberofResaleApplicationsRegisteredbyFlatType.csv')
hdb_resale_price_index = pd.read_csv('HousingAndDevelopmentBoardResalePriceIndex1Q2009100Quarterly.csv')
private_residential_price_index = pd.read_csv('PrivateResidentialPropertyPriceIndexBaseQuarter2009Q1100Quarterly.csv')
gdp_data = pd.read_csv('PerCapitaGNIAndPerCapitaGDPAtCurrentPricesAnnual.csv')



# Cleaning, filtering and transforming Datasets

In [105]:
# Transpose gdp data (horizontal to vertical)
gdp_data = gdp_data.transpose()
gdp_data.columns = gdp_data.iloc[0]  # Set the first row as the header
gdp_data = gdp_data.drop(gdp_data.index[0])  # Drop the first row
gdp_data = gdp_data.reset_index()
gdp_data.columns.name = None # Remove the column name
gdp_data.rename(columns={'index': 'year'}, inplace=True)

# Cleaning the data
# Function to check for missing values and print summary
def check_missing_values(df, name):
    # print(f"Summary for {name}:")
    # print(df.head())
    print(f"\nMissing values in {name}:")
    print(df.isnull().sum())
    print("\n")

# Check for missing values and print summaries
check_missing_values(private_sales, 'Private Sales')
check_missing_values(hdb_resale, 'HDB Resale')
check_missing_values(hdb_resale_price_index, 'HDB Resale Price Index')
check_missing_values(private_residential_price_index, 'Private Residential Price Index')
check_missing_values(gdp_data, 'GDP Data')

# # Example handling missing values for HDB Resale dataset
private_sales['sale_status'] = private_sales['sale_status'].replace('na', pd.NA)

# Replace NaN values with 'Total Applications' in the 'sale_status' column
private_sales['sale_status'] = private_sales['sale_status'].fillna('Applied')


# Transforming the data
# private_sales
# Pivot the DataFrame
pivot_df = private_sales.pivot_table(index='quarter', columns=['type_of_sale', 'sale_status'], values='units', aggfunc='sum')
# Flatten the MultiIndex columns
pivot_df.columns = ['private_' + '_'.join(col).lower().strip() if isinstance(col, tuple) else col.lower() for col in pivot_df.columns]
# Reset the index to turn the 'quarter' index back into a column
pivot_df = pivot_df.reset_index()
# Expected columns
expected_columns = ['quarter', 'private_new sale_completed', 'private_new sale_uncompleted', 'private_resale_applied', 'private_sub sale_applied']
# Reindex to ensure all expected columns are present, filling missing columns with 0
pivot_df = pivot_df.reindex(columns=expected_columns, fill_value=0)
# Assign the pivoted DataFrame back to the original DataFrame
private_sales=pivot_df
# print(private_sales.head())

# hdb_resale
# Pivot the DataFrame
pivot_df = hdb_resale.pivot_table(index='quarter', columns='flat_type', values='no_of_resale_applications', aggfunc='sum')
# Flatten the MultiIndex columns and convert to lowercase
pivot_df.columns = ['hdb_' + col.replace(' ', '_').replace("-", "_").lower() for col in pivot_df.columns]
# Reset the index to turn the 'quarter' index back into a column
pivot_df = pivot_df.reset_index()
# Expected columns
expected_columns = ['quarter', 'hdb_1_room', 'hdb_2_room', 'hdb_3_room', 'hdb_4_room', 'hdb_5_room', 'hdb_executive']
# Reindex to ensure all expected columns are present, filling missing columns with 0
pivot_df = pivot_df.reindex(columns=expected_columns, fill_value=0)
# Rename pivot_df back to hdb_resale
hdb_resale = pivot_df
# print(hdb_resale.head())

# private_residential_price_index
# Group by 'quarter' and calculate the mean of 'index'
private_residential_price_index= private_residential_price_index.groupby('quarter')['index'].mean().reset_index()
# Round the index values to 1 decimal place
private_residential_price_index['index'] = private_residential_price_index['index'].round(1)
# print(private_residential_price_index.head())

# Filtering and Joining the Datasets
# Merge datasets on quarter
merged_data = private_sales.merge(hdb_resale, on='quarter', how='inner')
merged_data = merged_data.merge(hdb_resale_price_index, on='quarter', how='inner')
merged_data = merged_data.merge(private_residential_price_index, on='quarter', how='inner', suffixes=('_hdb', '_private'))

# Extract year from quarter for GDP data merging
merged_data['year'] = merged_data['quarter'].str[:4].astype(int)

# Ensure the year column is of the same type in both DataFrames
gdp_data['year'] = gdp_data['year'].astype(str)
merged_data['year'] = merged_data['year'].astype(str)

# Merge GDP data based on year
merged_data = merged_data.merge(gdp_data, on='year', how='left')

# Drop the 'year' column as it's no longer needed
merged_data = merged_data.drop(columns=['year'])

# Print the resulting DataFrame
print(merged_data.head())




Missing values in Private Sales:
quarter         0
type_of_sale    0
sale_status     0
units           0
dtype: int64



Missing values in HDB Resale:
quarter                      0
flat_type                    0
no_of_resale_applications    0
dtype: int64



Missing values in HDB Resale Price Index:
quarter    0
index      0
dtype: int64



Missing values in Private Residential Price Index:
quarter          0
property_type    0
index            0
dtype: int64



Missing values in GDP Data:
year                          0
Per Capita GNI                0
Per Capita GDP                0
Per Capita GNI (US Dollar)    0
Per Capita GDP (US Dollar)    0
dtype: int64


   quarter  private_new sale_completed  private_new sale_uncompleted  \
0  2007-Q1                         218                          4565   
1  2007-Q2                         309                          4820   
2  2007-Q3                          83                          3367   
3  2007-Q4                          52  

# Nature of the Datasets

